# Introduction

This script was produced by Russell Tran for the [Guiyan Yan Lab](https://www.faculty.uci.edu/profile.cfm?faculty_id=5307) at UC Irvine. The Yan lab [studies](https://news.uci.edu/2017/04/27/9-6-million-grant-fuels-uci-malaria-control-research-in-africa/) the relationship between human-made environmental modifications (dams, irrigation, etc.) and malaria, and focuses on Kenya and Ethiopia.  

The script uses the [Opentrons](http://opentrons.com/) OT-2 robotics platform to automate Machery-Nagel's ["Genomic DNA from Blood"](https://www.mn-net.com/Portals/8/attachments/Redakteure_Bio/Protocols/Genomic%20DNA/UM_gDNABlood_NMag200.pdf) protocol (procedure), published Dec 2015. In brief, the protocol uses various buffers and Machery-Nagel's NucleoMag magnetic beads to isolate DNA from a 200uL blood sample. Should the Yan lab decide to invest in an OT-2 robot, this script would help the lab cut down on the 4-5 hours of manual labor they report it takes them to complete the protocol once (for each 96 samples). 

# Measurements / Notes for refining the robot's technique

(The protocol has been made to work with Qiagen troughs, though other troughs work too)
- The Qiagen 170 mL troughs have depth of about 52.5 mm, so going to 47 mm deep is a good idea
- The Qiagen 170 mL trough: 7mm height = 30 mL (RECTANGULAR PRISM VOLUME, not that of the trough indent on the bottom)
- Square_wells total depth from the top is 44 mm
- Square_wells meniscus 18 mm from bottom = 800 uL of volume (ratio is 0.0225 mm/uL)
- MBL_3 trough: 12 mm down is sufficient for first "drink" at 170 mL
- 80% Ethanol trough: 30 mm down down is sufficient for first "drink" at 80 mL

# Before starting the protocol:

- Check if Proteinase K was prepared according to section 3.

### Load the labware

In [ ]:
from opentrons import containers, instruments, robot, modules, labware

#Deck Slot 1 - MBL_2

MBL_2 = labware.load('point', '1', 'MBL_2')

#Deck Slot 2 - tipracks A for each step

tiprack_2a = containers.load('tiprack-200ul', '2', 'tiprack_2a')
tiprack_3a = containers.load('tiprack-200ul', '2', 'tiprack_3a', share=True)
tiprack_4a = containers.load('tiprack-200ul', '2', 'tiprack_4a', share=True)
tiprack_5a = containers.load('tiprack-200ul', '2', 'tiprack_5a', share=True)

#Deck Slot 3 - tipracks B for each step

tiprack_2b = containers.load('tiprack-200ul', '3', 'tiprack_2b')
tiprack_3b = containers.load('tiprack-200ul', '3', 'tiprack_3b', share=True)
tiprack_4b = containers.load('tiprack-200ul', '3', 'tiprack_4b', share=True)
tiprack_5b = containers.load('tiprack-200ul', '3', 'tiprack_5b', share=True)

#Deck Slot 4 - MBL_3

MBL_3 = labware.load('point', '4', 'MBL_3')

#Deck Slot 5 - EMPTY for now

#Deck Slot 6 - Elution Plate

elution_plate = labware.load('96-flat', '6', 'elution plate')

#Deck Slot 7 - 80% ethanol

ethanol = labware.load('point', '7', 'ethanol')

#Deck Slot 8 - Square Well Block (deep well block), with Magnetic Deck module underneath

magdeck = modules.load('magdeck', '8') # Note: will need to specify custom magnet engage height
square_wells = labware.load('96-deep-well', '8', 'square_wells', share=True)

#Deck Slot 9 - Chemical waste receptacle

chem_waste = labware.load('point', '9', 'chem_waste')

#Deck Slot 10 - MBL_4

MBL_4 = labware.load('point', '10', 'MBL_4')

#Deck Slot 11 - MBL_5

MBL_5 = labware.load('point', '11', 'MBL_5')

### Load the 8-Channel Pipette

In [ ]:
eight_pipettes = instruments.P300_Multi(
    tip_racks=[
        tiprack_2a,
        tiprack_2b,
        tiprack_3a,
        tiprack_3b,
        tiprack_4a,
        tiprack_4b
        tiprack_5a,
        tiprack_5b
    ],
    mount="left"
)

# Functions



In [ ]:
# trough_source - labware item to source washing fluid
# quantities_to_put_in - array that should add up to total volume want to put in,
#    but broken into 300 uL increments and smaller
# distances_when_putting_in - array whose length should be the same as quantities_to_put_in.
#    This is how many mm deep to go each time
def add_fluid(trough_source, 
              quantities_to_put_in,
              distances_when_putting_in
             ):
    
    columns = ['1','2','3','4','5','6','7','8','9','10','11','12']

    # for each column, use new pipette tips but keep looping with same tip until total qty dispensed for column.
    for column in columns: 
                            # NOTE: Current process does not follow GMP 
                            # (https://en.wikipedia.org/wiki/Good_manufacturing_practice)
                            # The code must be updated such that we should instead use new tips *every single* dispense,
                            # even if it requires the usage of 3x more pipette tips.
    
        eight_pipettes.pick_up_tip()
    
        for quantity, distance in zip(quantities_to_put_in, distances_when_putting_in):

            eight_pipettes.move_to(trough_source, strategy = "arc")

            eight_pipettes.aspirate(quantity, trough_source)

            eight_pipettes.move_to(square_wells.cols(column).top(distance), strategy = "arc")
    
            eight_pipettes.dispense(quantity, square_wells.cols(column).top(distance))
    
        eight_pipettes.mix(repetitions=15, volume=300, location=square_wells.cols(column).top(-10), rate=1.0)
    
        eight_pipettes.drop_tip()
    

    
# quantities_to_take_out - array that should add up to total volume want to take out,    
#    but broken into 300 uL increments and smaller for finesse
# distances_when_taking_out - array whose length should be the same as quantities_to_take_out.
#    This is how many mm deep to go each time
def remove_supernatant(quantities_to_take_out, 
                       distances_when_taking_out):
    
    columns = ['1','2','3','4','5','6','7','8','9','10','11','12']
    
    for column in columns: #for each column, use new pipette tips but keep looping same tips until total qty removed for column.
                    
        eight_pipettes.pick_up_tip()
    
        for quantity, distance in zip(quantities_to_take_out, distances_when_taking_out):

            eight_pipettes.move_to(square_wells.cols(column).top(distance), strategy = "arc")

            eight_pipettes.aspirate(quantity, square_wells.cols(column).top(distance))

            eight_pipettes.move_to(chem_waste['A1'], strategy = "arc")
    
            eight_pipettes.dispense(quantity, chem_waste['A1'])
        
        
        eight_pipettes.drop_tip()
        
        
# quantities_to_take_out - array that should add up to total volume want to take out,    
#    but broken into 300 uL increments and smaller for finesse
# distances_when_taking_out - array whose length should be the same as quantities_to_take_out.
#    This is how many mm deep to go each time
def transfer_supernatant(quantities_to_take_out, 
                       distances_when_taking_out): # NOTE: This function should be called for Step 7 ONLY
    
    columns = ['1','2','3','4','5','6','7','8','9','10','11','12']
    
    for column in columns: #for each column, use new pipette tips but keep looping same tips until total qty removed for column.
                    
        eight_pipettes.pick_up_tip()
    
        for quantity, distance in zip(quantities_to_take_out, distances_when_taking_out):

            eight_pipettes.move_to(square_wells.cols(column).top(distance), strategy = "arc")

            eight_pipettes.aspirate(quantity, square_wells.cols(column).top(distance))

            eight_pipettes.move_to(elution_plate[column], strategy = "arc")
    
            eight_pipettes.dispense(quantity, elution_plate[column])
        
        
        eight_pipettes.drop_tip()

# 1. Lyse samples (NOTE: Prepare Manually)

Dispense 20 uL of Proteinase K solution into each well of a Square-well Block.

Transfer 200 uL blood (equilibrated to room temperature) to each well of a Square-well Block. Do not moisten the rims of the well. 

Note: See recommendations for suitable plates or tubes and compatible magnetic separators (section 2.3). 

Add 80 uL Buffer MBL1 to each sample and mix by repeated pipetting up and down (3-5 times) and shaking for 5-10 min at room temperature. 

Alternatively, when processing the kit without a shaker, pipette up and down 10 times and incubate 5-10 min at room temperature. 

In [ ]:
# STEP 1 will be prepared MANUALLY, as the OT-2 is limited to 2 pipettes.

# 2. Bind DNA to NucleoMag B-Beads

Add 25 uL NucleoMag B-Beads to each sample. Mix magnetic  beads thoroughly before dispensing to the samples.

Add 300 uL Buffer MBL2 to each sample and mix by pipetting up and down 3-5 times and shake for 5 min to allow the DNA to bind to the magnetic beads. Alternatively, when processing the kit without a shaker, pipetting up and down 10 times and incubate 5 min at room temperature. 

Note: NucleoMag B-Beads and Buffer MBL2 can be premixed. For each sample to be processed, mix 25 uL of NucleoMag B-Beads with 300 uL Buffer MBL2. Vortex briefly. Depending on the dead volume of the reservior, additional amounts of bead suspension and binding buffer are necessary. Mix the solution several times to avoid the beads to settle within the premix distribution step. Do not store the premix of the NucleoMag B-Beads and Buffer MBL2 longer than 12 h.


Be sure to resuspend the NucleoMag B-Beads before removing them from the storage bottle. Vortex storage bottle briefly until a homogenous suspension has been formed. 

### IMPORTANT - For the purposes of this protocol, the MBL_2 trough contains the 300 uL:25 uL premixed together. Therefore, the above is summarized into the following:

325 uL of MBL_2+NucleoMag B-Beads needs to be pipetted into each square well and then mixed by pipette 15 (5+10=15) times

In [ ]:
add_fluid(trough_source = MBL_2.wells('A1').top(-47), 
          quantities_to_put_in = [250, 75],
          distances_when_putting_in = [-40, -34])

### Continuing on...
Separate the magnetic beads against the side of the wells by placing the Square-Well Block on the magnetic separator.

In [ ]:
# Engage MagDeck
magdeck.engage()

Wait at least 2 min until all the beads have been attracted by the magnet. 

In [ ]:
eight_pipettes.delay(min=3) # 3 minutes to be conservative

Remove and discard the supernatant by pipetting. Note: Do not disturb the attracted beads while aspirating the supernatant. The magnetic pellet is not visible in this step. Remove supernatant from the opposite side of the well.

In [ ]:
# Note: Arguments need to be set & refined
remove_supernatant(quantities_to_take_out, 
                   distances_when_taking_out)

# 3. Wash with MBL3 (1st wash)

Remove the Square-well Block from the magnetic separator.

Add 800 uL Buffer MBL3 to each well and resuspend the bead/DNA complex by shaking at room temperature until the beads are resuspended completely. Alternatively, resuspend the beads by pipetting up and down (15 times).

Note: Make sure that the magnetic beads are resuspended completely and form a brownish suspension. If necessary increase shaking incubation time or number of mixing cycles. Incomplete mixing may result in low purity of eluted DNA.


Separate the magnetic beads by placing the Square-well Block on the magnetic separator. Wait for at least 2 min until all the beads have been attracted to the magnet. Remove and discard supernatant by pipetting. 

Note: Supernatant has a brownish color, magnetic bead pellet is now visible.

In [ ]:
# Disengage MagDeck
magdeck.disengage()

# Note: Distances must be adjusted/refined to perfection
add_fluid(trough_source = MBL_3.wells('A1').top(-47), 
          quantities_to_put_in = [300,300, 200],
          distances_when_putting_in = [-10, -10, -10]
         )
    
# Engage MagDeck
magdeck.engage()

# Wait at least 2 min
eight_pipettes.delay(min=3)

# Remove supernatant - Note: arguments need to be set & refined
remove_supernatant(quantities_to_take_out, 
                   distances_when_taking_out)

# 4. Wash with MBL3 (2nd wash)

Remove the Square-well Block from the magnetic separator. 

Add 800 uL Buffer MBL3 to each well for a second wash step with Buffer MBL3. Wash the bead/DNA complex by shaking (5 min) at room temperature. Alternatively, resuspend the beads by pipetting up and down (15 times).

Separate the magnetic beads by placing the Square-well Block on the magnetic separator. Wait for at least 2 min until all the beads have been attracted to the magnet. Remove and discard supernatant by pipetting.

Note: Supernatant is colorless, magnetic bead pellet is clearly visible.

In [ ]:
# Disengage MagDeck
magdeck.disengage()

# Note: Distances must be adjusted/refined to perfection
add_fluid(trough_source = MBL_3.wells('A1').top(-47), 
          quantities_to_put_in = [300, 300, 200],
          distances_when_putting_in = [-10, -10, -10]
         )
    
# Engage MagDeck
magdeck.engage()

# Wait at least 2 min
eight_pipettes.delay(min=3)

# Remove supernatant - Note: arguments need to be set & refined
remove_supernatant(quantities_to_take_out, 
                   distances_when_taking_out)

# 5. Wash with 80% ethanol

Remove the Square-Well Block from the magnetic separator. 

Add 800 uL 80% ethanol to each well and wash the bead/DNA complex by shaking (5 min) at room temperature. Alternatively, resuspend the beads by pipetting up and down (15 times).

Separate the magnetic beads by placing the Square-Well Block on the magnetic separator. Wait for at least 2 min until all the beads have been attracted to the magnet. Remove and discard supernatant by pipetting. 

Note: Supernatant is colorless, magnetic bead pellet is visible now. 

In [ ]:
# Disengage MagDeck
magdeck.disengage()

# Note: Distances must be adjusted
add_fluid(trough_source = ethanol.wells('A1').top(-47), 
          quantities_to_put_in = [300, 300, 200],
          distances_when_putting_in = [-10, -10, -10]
         )
    
# Engage MagDeck
magdeck.engage()

# Wait at least 2 min
eight_pipettes.delay(min=3)

# Remove supernatant - Note: arguments need to be set & refined
remove_supernatant(quantities_to_take_out, 
                   distances_when_taking_out)

# 6. Wash with MBL4

Leave Square-well Block on the magnetic separator.

***IMPORTANT*** Gently add 900 uL Buffer MBL4 to each well and incubate for 45-90 s while the beads are still attracted to the magnet. 

Then aspirate and discard the supernatant.

Note: Do **NOT** resuspend the beads in Buffer MBL4. This step is to remove traces of ethanol and eliminates a drying step.

Optional: Washing the magnetic beads with Buffer MBL4 may decrease the DNA yield slightly. Alternatively, replace this washing step by air-drying of the magnetic beads for 10-15 min until all of the ethanol from previous washing step has evaporated. beads with remaining ethanol appear to be glossy. Moderate heating (37 degrees C) can support and shorten the air-drying step. Over drying the beads may result in low yield in the final elution step.

In [ ]:
# Disengage MagDeck
magdeck.disengage()

# Note: Distances must be adjusted, especially because this step requires a gentle dispense (dispense speed 0.5?)
add_fluid(trough_source = MBL_4.wells('A1').top(-47), 
          quantities_to_put_in = [300, 300, 300],
          distances_when_putting_in = [-10, -10, -10]
         )
    
# Engage MagDeck
magdeck.engage()

# Wait at least 2 min
eight_pipettes.delay(min=3)

# Remove supernatant - Note: arguments need to be set & refined
remove_supernatant(quantities_to_take_out, 
                   distances_when_taking_out)

# 7. Elute DNA

Remove the Square-well Block from the magnetic separator.

Add desired volume of Buffer MBL5 (50-100 uL) to each well of the Square-Well Block and resuspend the bead/DNA complex by shaking (5-10 min). Alternatively, resuspend the beads by pipetting up and down (15 times). 

Separate the magnetic beads by placing the Square-well Block on the magnetic separator. Wait for at least 2 min until all the beads have been attracted to the magnet. Transfer the supernatant containing the purified genomic DNA to the Elution Plate. 

Note: Yield can be increase by 15-20% by using pre-heated elution buffer (55-72 degrees C) or by incubating the bead/elution buffer suspension at 55-72 degrees C for to 10 min. 

In [ ]:
# Disengage MagDeck
magdeck.disengage()

# Note: Distances must be adjusted/refined
add_fluid(trough_source = MBL_5.wells('A1').top(-47), 
          quantities_to_put_in = [75, 75, 75],
          distances_when_putting_in = [-10, -10, -10]
         )
    
# Engage MagDeck
magdeck.engage()

# Wait at least 2 min
eight_pipettes.delay(min=3)

# Transfer supernatant - Note: arguments need to be set & refined
transfer_supernatant(quantities_to_take_out, 
                       distances_when_taking_out)